# Build Data for the Map

Check and merge datasets from:
    cordis
    creative
    esif
    fts
    erasmus
    nhs
    nweurope
    life

In [2]:
%matplotlib inline

from functools import reduce
import glob
import json
import os

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)

## Define Validity Checks

In [3]:
ukpostcodes = pd.read_csv('../postcodes/input/ukpostcodes.csv.gz')
ukpostcodes.shape

(1762397, 4)

In [4]:
def validate_postcodes(df):
    assert 'postcode' in df.columns
    assert (~df['postcode'].isin(ukpostcodes.postcode)).sum() == 0
    
def validate_date_range(df):
    assert 'start_date' in df.columns
    assert 'end_date' in df.columns
    assert df['start_date'].dtype == 'datetime64[ns]'
    assert df['end_date'].dtype == 'datetime64[ns]'
    assert (df['start_date'] > df['end_date']).sum() == 0

## Load Cleaned Data

### CORDIS

In [5]:
fp7_organizations = pd.read_pickle('../cordis/output/fp7_organizations.pkl.gz')
validate_postcodes(fp7_organizations)
fp7_organizations.head()

,project_rcn,organization_id,role,name,contribution_eur,organization_url,postcode
0,85234,999903840,participant,The University of Manchester,346277.0,http://www.manchester.ac.uk,M13 9PL
1,85235,999974165,participant,University of Glasgow,616725.0,http://www.gla.ac.uk,G12 8QQ
2,85236,999975426,participant,University of Leeds,129120.0,http://www.leeds.ac.uk,LS2 9JT
3,85236,999990752,participant,Loughborough University,74835.0,http://www.lboro.ac.uk,LE11 3TU
4,85238,999840984,participant,Lancaster University,414167.0,http://www.lancaster.ac.uk,LA1 4YW


In [6]:
fp7_projects = pd.read_pickle('../cordis/output/fp7_projects.pkl.gz')
validate_date_range(fp7_projects)
fp7_projects.head()

,rcn,acronym,status,title,start_date,end_date,project_url,objective,total_cost_eur,max_contribution_eur,coordinator_country,participant_countries,brief_title,teaser,article,related_report_title,image_path,num_countries,num_organizations,eur_gbp
0,210895,MODERN,ONG,"MOdeling and DEsign of Reliable, process varia...",2009-03-01,2012-02-01,NaN,The influence of process variations is becomin...,2.736301e+07,4385254.00,FR,ES;DK;AT;FR;CH;EL;IT;NL;DE;UK,NaN,NaN,NaN,NaN,NaN,10,26,0.871217
1,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050
2,210444,E450LMDAP,ONG,European 450mm Lithography and Metrology Devel...,2014-10-01,2017-10-01,NaN,The overall goal of the E450LMDAP project is t...,4.593065e+08,68895968.66,NL,NL;FR;DE;IL;BE;UK,NaN,NaN,NaN,NaN,NaN,6,41,0.795859
3,208270,GENIUS,ONG,GNSS Education Network for Universities and In...,2012-01-04,2016-10-03,NaN,The GENIUS project has been designed to build ...,1.093128e+06,844912.00,UK,DE;FR;ES;IT,NaN,NaN,NaN,NaN,NaN,5,9,0.797839
4,208281,G5-AsiA,ONG,GNSS for Asia - Support on International Activ...,2012-01-01,2014-06-30,NaN,The G4-AsiA project will support the European...,1.347931e+06,1248701.00,DE,CN;BE;JP;UK;FR,NaN,NaN,NaN,NaN,NaN,6,6,0.829205


In [7]:
fp7 = pd.merge(
    fp7_projects, fp7_organizations,
    left_on='rcn', right_on='project_rcn', validate='1:m'
)
fp7['my_eu_id'] = 'fp7_' + fp7.project_rcn.astype('str') + '_' + fp7.organization_id.astype('str')
fp7.head()

,rcn,acronym,status,title,start_date,end_date,project_url,objective,total_cost_eur,max_contribution_eur,coordinator_country,participant_countries,brief_title,teaser,article,related_report_title,image_path,num_countries,num_organizations,eur_gbp,project_rcn,organization_id,role,name,contribution_eur,organization_url,postcode,my_eu_id
0,210895,MODERN,ONG,"MOdeling and DEsign of Reliable, process varia...",2009-03-01,2012-02-01,NaN,The influence of process variations is becomin...,2.736301e+07,4385254.00,FR,ES;DK;AT;FR;CH;EL;IT;NL;DE;UK,NaN,NaN,NaN,NaN,NaN,10,26,0.871217,210895,999974165,participant,University of Glasgow,64128.0,http://www.gla.ac.uk,G12 8QQ,fp7_210895_999974165
1,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,966670282,participant,British Gas Trading Limited,14610.0,http://www.britishgas.co.uk,SL4 5GD,fp7_210893_966670282
2,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,999953504,participant,Qinetiq Limited,232464.0,NaN,GU14 0LX,fp7_210893_999953504
3,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,999977172,participant,The Chancellor Masters and Scholars of the Uni...,74189.0,http://www.cam.ac.uk,CB2 1TN,fp7_210893_999977172
4,210444,E450LMDAP,ONG,European 450mm Lithography and Metrology Devel...,2014-10-01,2017-10-01,NaN,The overall goal of the E450LMDAP project is t...,4.593065e+08,68895968.66,NL,NL;FR;DE;IL;BE;UK,NaN,NaN,NaN,NaN,NaN,6,41,0.795859,210444,959334463,participant,Oxford Instruments Nanotechnology Tools Limited,162015.0,NaN,OX13 5QX,fp7_210444_959334463


In [8]:
h2020_organizations = pd.read_pickle('../cordis/output/h2020_organizations.pkl.gz')
validate_postcodes(h2020_organizations)
h2020_organizations.head()

,project_rcn,organization_id,role,name,contribution_eur,organization_url,postcode
0,193157,999905974,coordinator,The Wellcome Trust Limited,230000.0,http://www.wellcome.ac.uk,NW1 2BE
1,193159,998811911,participant,United Kingdom Atomic Energy Authority,27479945.0,http://www.ukaea.org.uk,OX14 3DB
2,193160,999574428,participant,Cambridge Environmental Research Consultants Ltd,20000.0,NaN,CB2 1SJ
3,193160,999916741,coordinator,European Centre for Medium-Range Weather Forec...,1190000.0,http://www.ecmwf.int,RG2 9AX
4,193160,999975426,participant,University of Leeds,68662.0,http://www.leeds.ac.uk,LS2 9JT


In [9]:
h2020_projects = pd.read_pickle('../cordis/output/h2020_projects.pkl.gz')
validate_date_range(h2020_projects)
h2020_projects.head()

,rcn,acronym,status,title,start_date,end_date,project_url,objective,total_cost_eur,max_contribution_eur,coordinator_country,participant_countries,num_countries,num_organizations,eur_gbp
0,198301,BlueBRIDGE,SIGNED,Building Research environments for fostering I...,2015-09-01,2018-02-28,NaN,BlueBRIDGE responds to the Juncker Investment ...,5295753.75,5295753.75,IT,IT;EL;NO;FR;DK;UK,6,14,0.828705
1,195912,NeuroEE,SIGNED,Delineation of a brain circuit regulating ener...,2016-02-01,2018-01-31,NaN,Obesity is now a global epidemic with a signif...,195454.80,195454.80,UK,NaN,1,1,0.848325
2,207022,INSHIP,SIGNED,Integrating National Research Agendas on Solar...,2017-01-01,2020-12-31,NaN,Despite process heat is recognized as the appl...,2858798.75,2498661.25,DE,ES;FR;IT;PT;EL;AT;BE;DE;CH;UK;CY;TR,12,29,0.872316
3,210501,DELTA-FLU,SIGNED,Dynamics of avian influenza in a changing world,2017-06-01,2022-05-31,https://delta-flu.fli.de/,"DELTA-FLU aims to determine the key viral, hos...",5497793.75,5497793.75,DE,SE;UK;BE;US;IT;HK;NL,8,9,0.873419
4,203271,GROW,SIGNED,GROW Observatory,2016-11-01,2019-10-31,NaN,The GROW Observatory (GROW) will create a sust...,5379290.58,5096919.64,UK,IT;AT;IE;FR;UK;NL;HU;EL;BE;ES,10,19,0.873137


In [10]:
h2020 = pd.merge(
    h2020_projects, h2020_organizations,
    left_on='rcn', right_on='project_rcn', validate='1:m'
)
h2020['my_eu_id'] = 'h2020_' + h2020.project_rcn.astype('str') + '_' + h2020.organization_id.astype('str')

# no briefs available for H2020
h2020['related_report_title'] = float('nan')
h2020['brief_title'] = float('nan')
h2020['teaser'] = float('nan')
h2020['article'] = float('nan')
h2020['image_path'] = float('nan')

h2020.head()

,rcn,acronym,status,title,start_date,end_date,project_url,objective,total_cost_eur,max_contribution_eur,coordinator_country,participant_countries,num_countries,num_organizations,eur_gbp,project_rcn,organization_id,role,name,contribution_eur,organization_url,postcode,my_eu_id,related_report_title,brief_title,teaser,article,image_path
0,198301,BlueBRIDGE,SIGNED,Building Research environments for fostering I...,2015-09-01,2018-02-28,NaN,BlueBRIDGE responds to the Juncker Investment ...,5295753.75,5295753.75,IT,IT;EL;NO;FR;DK;UK,6,14,0.828705,198301,999687724,participant,Trust-It Services Limited,350860.0,NaN,EN2 6NF,h2020_198301_999687724,NaN,NaN,NaN,NaN,NaN
1,195912,NeuroEE,SIGNED,Delineation of a brain circuit regulating ener...,2016-02-01,2018-01-31,NaN,Obesity is now a global epidemic with a signif...,195454.80,195454.80,UK,NaN,1,1,0.848325,195912,999929448,coordinator,The University Court of the University of Aber...,195454.8,http://www.abdn.ac.uk,AB24 3FX,h2020_195912_999929448,NaN,NaN,NaN,NaN,NaN
2,207022,INSHIP,SIGNED,Integrating National Research Agendas on Solar...,2017-01-01,2020-12-31,NaN,Despite process heat is recognized as the appl...,2858798.75,2498661.25,DE,ES;FR;IT;PT;EL;AT;BE;DE;CH;UK;CY;TR,12,29,0.872316,207022,999440762,participant,Cranfield University,10000.0,http://www.cranfield.ac.uk,MK43 0AL,h2020_207022_999440762,NaN,NaN,NaN,NaN,NaN
3,210501,DELTA-FLU,SIGNED,Dynamics of avian influenza in a changing world,2017-06-01,2022-05-31,https://delta-flu.fli.de/,"DELTA-FLU aims to determine the key viral, hos...",5497793.75,5497793.75,DE,SE;UK;BE;US;IT;HK;NL,8,9,0.873419,210501,999904713,participant,"The Secretary of State for Environment, Food a...",687500.0,NaN,SW1P 3JR,h2020_210501_999904713,NaN,NaN,NaN,NaN,NaN
4,210501,DELTA-FLU,SIGNED,Dynamics of avian influenza in a changing world,2017-06-01,2022-05-31,https://delta-flu.fli.de/,"DELTA-FLU aims to determine the key viral, hos...",5497793.75,5497793.75,DE,SE;UK;BE;US;IT;HK;NL,8,9,0.873419,210501,999974941,participant,The University of Edinburgh,524385.0,http://www.ed.ac.uk,EH8 9YL,h2020_210501_999974941,NaN,NaN,NaN,NaN,NaN


In [11]:
assert set(fp7.columns) == set(h2020.columns)

In [12]:
cordis = pd.concat([fp7, h2020[fp7.columns]])
cordis.shape

(27293, 28)

In [13]:
cordis['total_cost_gbp'] = (cordis.total_cost_eur * cordis.eur_gbp).round()
cordis['max_contribution_gbp'] = (cordis.max_contribution_eur * cordis.eur_gbp).round()
cordis['contribution_gbp'] = (cordis.contribution_eur * cordis.eur_gbp).round()
cordis.head()

,rcn,acronym,status,title,start_date,end_date,project_url,objective,total_cost_eur,max_contribution_eur,coordinator_country,participant_countries,brief_title,teaser,article,related_report_title,image_path,num_countries,num_organizations,eur_gbp,project_rcn,organization_id,role,name,contribution_eur,organization_url,postcode,my_eu_id,total_cost_gbp,max_contribution_gbp,contribution_gbp
0,210895,MODERN,ONG,"MOdeling and DEsign of Reliable, process varia...",2009-03-01,2012-02-01,NaN,The influence of process variations is becomin...,2.736301e+07,4385254.00,FR,ES;DK;AT;FR;CH;EL;IT;NL;DE;UK,NaN,NaN,NaN,NaN,NaN,10,26,0.871217,210895,999974165,participant,University of Glasgow,64128.0,http://www.gla.ac.uk,G12 8QQ,fp7_210895_999974165,23839127.0,3820510.0,55869.0
1,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,966670282,participant,British Gas Trading Limited,14610.0,http://www.britishgas.co.uk,SL4 5GD,fp7_210893_966670282,22921264.0,3827854.0,12361.0
2,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,999953504,participant,Qinetiq Limited,232464.0,NaN,GU14 0LX,fp7_210893_999953504,22921264.0,3827854.0,196676.0
3,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,2.709210e+07,4524384.00,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,999977172,participant,The Chancellor Masters and Scholars of the Uni...,74189.0,http://www.cam.ac.uk,CB2 1TN,fp7_210893_999977172,22921264.0,3827854.0,62768.0
4,210444,E450LMDAP,ONG,European 450mm Lithography and Metrology Devel...,2014-10-01,2017-10-01,NaN,The overall goal of the E450LMDAP project is t...,4.593065e+08,68895968.66,NL,NL;FR;DE;IL;BE;UK,NaN,NaN,NaN,NaN,NaN,6,41,0.795859,210444,959334463,participant,Oxford Instruments Nanotechnology Tools Limited,162015.0,NaN,OX13 5QX,fp7_210444_959334463,365543340.0,54831501.0,128941.0


In [14]:
cordis.describe()

,rcn,total_cost_eur,max_contribution_eur,num_countries,num_organizations,eur_gbp,project_rcn,organization_id,contribution_eur,total_cost_gbp,max_contribution_gbp,contribution_gbp
count,27293.000000,2.716700e+04,2.729300e+04,27293.000000,27293.000000,27293.000000,27293.000000,2.729300e+04,2.575900e+04,2.716700e+04,2.729300e+04,2.575900e+04
mean,142187.446745,5.741565e+06,4.080762e+06,6.092441,12.386986,0.835107,142187.446745,9.912789e+08,4.409178e+05,4.793293e+06,3.413651e+06,3.691643e+05
std,51228.848231,1.204047e+07,7.145162e+06,4.582503,13.841218,0.032097,51228.848231,2.025782e+07,8.523053e+05,1.002322e+07,6.012937e+06,7.178402e+05
min,85234.000000,1.620000e+04,1.324800e+04,1.000000,1.000000,0.707313,85234.000000,8.888978e+08,1.243800e+02,1.310400e+04,1.048300e+04,1.030000e+02
25%,97826.000000,1.214142e+06,9.981470e+05,2.000000,3.000000,0.809893,97826.000000,9.983990e+08,1.435067e+05,1.011139e+06,8.376550e+05,1.196415e+05
50%,108767.000000,3.231279e+06,2.635228e+06,6.000000,10.000000,0.840899,108767.000000,9.999038e+08,2.427644e+05,2.701332e+06,2.197303e+06,2.028120e+05
75%,199339.000000,5.996175e+06,4.949337e+06,8.000000,16.000000,0.864473,199339.000000,9.999770e+08,4.797790e+05,5.047679e+06,4.112934e+06,4.001365e+05
max,216437.000000,8.569619e+08,4.408000e+08,46.000000,163.000000,0.896200,216437.000000,9.999935e+08,6.191296e+07,7.143460e+08,3.674419e+08,5.293806e+07


In [15]:
(cordis.contribution_eur > cordis.total_cost_eur + 0.1).sum()

0

In [16]:
[
    cordis.start_date.isna().sum(),
    cordis.end_date.isna().sum()
]

[0, 0]

### Creative Europe

In [17]:
creative_organisations = pd.read_pickle('../creative/output/creative_europe_organisations.pkl.gz')
creative_organisations.shape

(320, 10)

In [18]:
creative_projects = pd.read_pickle('../creative/output/creative_europe_projects.pkl.gz')
creative_projects.shape

(266, 21)

In [19]:
creative = pd.merge(creative_projects, creative_organisations, on='project_number', validate='1:m')
creative.shape

(320, 30)

In [20]:
validate_postcodes(creative)
validate_date_range(creative)
creative['max_contribution_gbp'] = (creative.max_contribution_eur * creative.eur_gbp).round()
creative['my_eu_id'] = \
    'creative_' + creative.project_number + '_' + \
    creative.partner_number.apply('{:.0f}'.format).\
    str.replace('nan', 'coordinator', regex=False)
assert creative.shape[0] == creative.my_eu_id.unique().shape[0]
creative.head()

,project_number,funds,category,action,activity_type,call_year,start_date,end_date,project,summary,project_status,max_contribution_eur,is_success,project_url,results_available,results_url,participating_countries,extra_partners,num_countries,num_organisations,eur_gbp,organisation_address,organisation_coordinator,organisation_country,organisation_name,partner_number,postcode,organisation_region,organisation_type,organisation_website,max_contribution_gbp,my_eu_id
0,552179-CREA-1-2014-1-IT-CULT-COOP1,Creative Europe,Culture,Cooperation projects,NaN,2014,2014-11-03,2016-12-15,Virtual Sets: Creating and promoting virtual s...,ViSet will promote cutting-edge digital techno...,finalized,199440.70,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"IT,ES,UK,RS",NaN,4,5,0.769165,BASECAMP LIVERPOOL UNIT N BALTIC CREATIVE CAMP...,False,UK,PACIFICSTREAM ENTERPRISE SOLUTIONSCOMMUNITY IN...,2.0,L1 0AH,Merseyside,Other,http://pacifcstreamenterprise.co.uk,153403.0,creative_552179-CREA-1-2014-1-IT-CULT-COOP1_2
1,552036-CREA-1-2014-1-IT-CULT-COOP2,Creative Europe,Culture,Cooperation projects,NaN,2014,2014-10-01,2018-09-30,"SMALL SIZE, PERFORMING ARTS FOR EARLY YEARS",The project has a precise focus: the diffusion...,ongoing,1995000.00,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"IT,IE,DE,RO,HU,SE,FR,SI,DK,ES,UK,PL,AT,FI,BE",NaN,15,17,0.816518,"240 THE BROADWAY, SW19 1SB, LONDON",False,UK,POLKA CHILDRENS THEATRE LIMITED,11.0,SW19 1SB,Extra-Regio NUTS 2,Theatre,http://www.polkatheatre.com,1628953.0,creative_552036-CREA-1-2014-1-IT-CULT-COOP2_11
2,552170-CREA-1-2014-1-ES-CULT-COOP1,Creative Europe,Culture,Cooperation projects,NaN,2014,2014-10-01,2016-12-31,Made@EU,Digital divide is currently hindering the wide...,finalized,200000.00,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"ES,UK,FR",NaN,3,3,0.771257,"TAVISTOCK PLACE, PL4 8AT, PLYMOUTH",False,UK,PLYMOUTH COLLEGE OF ART,1.0,PL4 8AT,Devon,Higher education institution (tertiary level),http://www.plymouthart.ac.uk,154251.0,creative_552170-CREA-1-2014-1-ES-CULT-COOP1_1
3,552219-CREA-1-2014-1-IE-CULT-COOP2,Creative Europe,Culture,Cooperation projects,"Theatre, Dance, Music, Street art",2014,2014-11-12,2018-09-15,COLLAB ARTS PARTNERSHIP PROGRAMME: Art in soci...,The COLLAB Arts Partnership Programme (CAPP) w...,ongoing,1425030.49,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"IE,DE,UK,FI,ES,HU",NaN,6,9,0.816919,"MILLBANK, SW1P 4RG, LONDON",False,UK,THE BOARD OF TRUSTEES OF THE TATE GALLERY,2.0,SW1P 4RG,Extra-Regio NUTS 2,Art association,http://www.tate.org.uk/,1164135.0,creative_552219-CREA-1-2014-1-IE-CULT-COOP2_2
4,552219-CREA-1-2014-1-IE-CULT-COOP2,Creative Europe,Culture,Cooperation projects,"Theatre, Dance, Music, Street art",2014,2014-11-12,2018-09-15,COLLAB ARTS PARTNERSHIP PROGRAMME: Art in soci...,The COLLAB Arts Partnership Programme (CAPP) w...,ongoing,1425030.49,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"IE,DE,UK,FI,ES,HU",NaN,6,9,0.816919,THE WHITE BUILDING UNIT 7 QUEENS YARD WHITE PO...,False,UK,LIVE ART DEVELOPMENT AGENCY,3.0,E9 5EN,Inner London,Art association,http://www.thisisliveart.co.uk,1164135.0,creative_552219-CREA-1-2014-1-IE-CULT-COOP2_3


In [21]:
creative.results_available.value_counts()

No     297
Yes     23
Name: results_available, dtype: int64

In [22]:
creative.results_url[0]

'https://ec.europa.eu/programmes/creative-europe/projects/ce-project-details/#project/552179-CREA-1-2014-1-IT-CULT-COOP1'

In [23]:
[creative.start_date.isna().sum(), creative.end_date.isna().sum()]

[0, 0]

### ESIF (ESF/ERDF)

In [82]:
esif_england = pd.read_pickle('../esif/output/esif_england_2014_2020.pkl.gz')
validate_postcodes(esif_england)
validate_date_range(esif_england)
esif_england.head()

,beneficiary,project,funds,summary,start_date,end_date,eu_investment,project_cost,postcode,my_eu_id
0,Building Research Establishment Limited,Hertfordshire Low Carbon Innovation Network,ERDF,Strategic Fit and Value for Money requirements...,2016-05-01,2019-02-28,630000.0,1260000.0,WD25 9XX,erdf_england_0
1,GCGP LEP,Promoting Growth in Greater Cambridge Greater ...,ERDF,The project will provide local facilitation re...,2015-10-01,2018-12-31,373677.0,747356.0,PE28 4WX,erdf_england_1
2,Satellite Applications Catapult,Westcott Business Incubation Centre,ERDF,Incubation centre for collaboration & business...,2017-10-01,2020-12-31,500000.0,1000000.0,OX11 0QR,erdf_england_2
3,Hastings Borough Council,CHART (Connecting Hastings and Rother Together...,ERDF,"Promoting self-employment, small business star...",2016-10-26,2022-12-31,2073538.0,4147076.0,TN34 3UY,erdf_england_3
4,Ngage Solutions,High Achieving Local Organisations (HALO),ERDF,Strategic Fit and value for Money requirements...,2016-09-01,2019-06-30,500000.0,1000000.0,HP14 4BF,erdf_england_4


In [83]:
esif_ni = pd.read_pickle('../esif/output/esif_ni_2014_2020.pkl.gz')
validate_postcodes(esif_ni)
validate_date_range(esif_ni)
esif_ni.head()

,beneficiary,eu_investment,project,project_cost,postcode,funds,start_date,end_date,summary,my_eu_id
0,CLANRYE GROUP LIMITED,1672213.0,Family Foundations,2572636.0,BT35 8SW,ESF,2014-01-01,2021-01-01,None,esf_ni_0
1,CUSTOMIZED TRAINING SERVICES,1558411.0,Community Family Support Programme,2397556.0,BT82 8AS,ESF,2014-01-01,2021-01-01,None,esf_ni_1
2,DFPF LTD T/A PEOPLE 1ST,1056824.0,Community Family Support Programme,1625884.0,BT9 6SP,ESF,2014-01-01,2021-01-01,None,esf_ni_2
3,NETWORK PERSONNEL (NP),2998933.0,Community Family Support Programme (CFSP),4613745.0,BT45 5AJ,ESF,2014-01-01,2021-01-01,None,esf_ni_3
4,UPPER SPRINGFIELD DEVELOPMENT TRUST,1351746.0,CFSP Family First Belfast,2079610.0,BT12 7FP,ESF,2014-01-01,2021-01-01,None,esf_ni_4


In [84]:
esif_scotland = pd.read_pickle('../esif/output/esif_scotland.pkl.gz')
validate_postcodes(esif_scotland)
validate_date_range(esif_scotland)
esif_scotland.head()

,beneficiary,project,summary,eu_investment,project_cost,funds,postcode,start_date,end_date,my_eu_id
1,Transport Scotland,LCTT Active travel/Low carbon Hubs CF LUPS,Challenge fund to develop 4 active / low carbo...,4948087.48,12470810.21,ERDF,G4 0HF,2017-05-24,2020-12-30,erdf_scotland_1
2,Transport Scotland,LCTT Active travel/Low carbon Hubs H&I,Challenge fund to develop 2 active / low carbo...,2409807.09,3442581.56,ERDF,G4 0HF,2016-05-02,2018-12-30,erdf_scotland_2
3,Transport Scotland,SI Management Costs,Management Costs,132524.70,331311.77,ERDF,G4 0HF,2016-03-01,2018-12-31,erdf_scotland_3
4,Transport Scotland,SI Management Costs - H&I,Management Costs,55218.63,110437.27,ERDF,G4 0HF,2016-03-01,2018-12-31,erdf_scotland_4
5,Transport Scotland,LCTT CF Management costs LUPS,Setting up and delivery of challenge fund,450000.00,1125000.00,ERDF,G4 0HF,2016-05-02,2018-12-31,erdf_scotland_5


In [85]:
esif_wales = pd.read_pickle('../esif/output/esif_wales.pkl.gz')
validate_postcodes(esif_wales)
validate_date_range(esif_wales)
esif_wales.head()

,project,funds,summary,beneficiary,start_date,end_date,eu_investment,project_cost,postcode,my_eu_id
0,SMARTInnovation (East Wales),ERDF,Financial support for Welsh businesses to driv...,"WG - Department for Economy, Science and Trans...",2015-02-09,2023-06-30,3779021.0,7558043.0,CF10 3NQ,erdf_wales_80751
1,Ser Cymru II (East Wales),ERDF,Attracting world class expertise to deliver cu...,"WG - Department for Economy, Science and Trans...",2015-11-09,2023-01-31,8000000.0,18473118.0,CF10 3NQ,erdf_wales_80762
2,SMARTCymru (East Wales),ERDF,Providing support for businesses at different ...,"WG - Department for Economy, Science and Trans...",2014-01-01,2021-09-30,12259800.0,29400000.0,CF10 3NQ,erdf_wales_80825
3,FLEXIS East,ERDF,"World leading energy research scheme, involvin...",Cardiff University,2015-07-01,2021-02-28,5621112.0,9827770.0,CF10 3AT,erdf_wales_80836
4,Cardiff University Brain Research Imaging Cent...,ERDF,"CUBRIC II is the relocation, expansion and dev...",Cardiff University,2014-11-28,2021-10-31,4578474.0,16156949.0,CF10 3AT,erdf_wales_80838


In [86]:
assert set(esif_england.columns) == set(esif_ni.columns)
assert set(esif_england.columns) == set(esif_scotland.columns)
assert set(esif_england.columns) == set(esif_wales.columns)
esif_columns = esif_england.columns
esif = pd.concat([
    esif_england,
    esif_ni[esif_columns],
    esif_scotland[esif_columns],
    esif_wales[esif_columns]
])
esif.shape

(1351, 10)

In [87]:
[esif.start_date.isna().sum(), esif.end_date.isna().sum()]

[0, 0]

### FTS

In [30]:
fts_2016 = pd.read_pickle('../fts/output/fts_2016.pkl.gz')
validate_postcodes(fts_2016)
fts_2016['year'] = 2016
fts_2016.head()

,beneficiary,coordinator,amount,geographical_zone,expense_type,total_amount_eur,subject_of_grant_or_contract,responsible_department,budget_line_name_and_number,action_type,funding_type,postcode,eur_gbp,my_eu_id,year
339,NATS (EN ROUTE) PUBLIC LIMITED COMPANY*NERL,yes,7.450001e+07,NaN,Operational,74500011.05,2015-EU-TM-0193-M - NO MOVE/E2-2014-717/SESAR...,Innovation and Networks Executive Agency,Optimising the integration and interconnection...,Connecting Europe Facility (CEF),Grants,PO15 7FL,0.811893,fts_2016_339,2016
367,UNITED KINGDOM ATOMIC ENERGY AUTHORITY*UKAEA,no,4.673079e+06,NaN,Administrative,72239003.00,633053 - EUROFUSION - IMPLEMENTATION OF ACTIVI...,Directorate-General for Research and Innovation,Euratom — Fusion energy (08.03.01.01),Euratom Research and Training Programme,Grants,OX14 3DB,0.811893,fts_2016_367,2016
374,UNITED KINGDOM ATOMIC ENERGY AUTHORITY*UKAEA,yes,6.000000e+07,NaN,Operational,60000000.00,NEW JOC CONTRACT FOR OPERATION OF JET FACILITI...,Directorate-General for Research and Innovation,Euratom — Fusion energy (08.03.01.01),Euratom Research and Training Programme,Grants,OX14 3DB,0.811893,fts_2016_374,2016
800,NATS (EN ROUTE) PUBLIC LIMITED COMPANY*NERL,yes,2.837278e+07,NaN,Operational,28372781.20,2015-EU-TM-0197-M INEA/CEF/TRAN/M2015/1132963...,Innovation and Networks Executive Agency,Connecting Europe Facility (CEF) — Cohesion Fu...,Cohesion Fund (CF),Grants,PO15 7FL,0.811893,fts_2016_800,2016
1128,INTERNATIONAL RESCUE COMMITTEE UK*IRC UK,yes,1.650000e+07,NaN,Operational,16500000.00,PROVISION OF EMERGENCY HUMANITARIAN ASSISTANCE...,Directorate-General for Humanitarian Aid and C...,Emergency support within the Union (18.07.01),Instrument for Emergency Support within the Union,Grants,WC1A 2QL,0.811893,fts_2016_1128,2016


In [31]:
fts_2017 = pd.read_pickle('../fts/output/fts_2017.pkl.gz')
validate_postcodes(fts_2017)
fts_2017['year'] = 2017
fts_2017.head()

,beneficiary,coordinator,amount,geographical_zone,expense_type,total_amount_eur,subject_of_grant_or_contract,responsible_department,budget_line_name_and_number,action_type,funding_type,postcode,eur_gbp,my_eu_id,year
17,THE MANCHESTER AIRPORT GROUP PLC*MAAG,no,3.245067e+06,NaN,Administrative,1.582960e+08,2016-EU-TM-0117-M - INEA/CEF/TRAN/M2016/134961...,Innovation and Networks Executive Agency,Optimising the integration and interconnection...,Connecting Europe Facility (CEF),Grants,M90 1QX,0.872252,fts_2017_17,2017
37,BRITISH AIRWAYS PLC*,no,6.331839e+05,NaN,Administrative,1.582960e+08,2016-EU-TM-0117-M - INEA/CEF/TRAN/M2016/134961...,Innovation and Networks Executive Agency,Optimising the integration and interconnection...,Connecting Europe Facility (CEF),Grants,UB7 0GB,0.872252,fts_2017_37,2017
42,EASYJET AIRLINE COMPANY LIMITED*,no,6.331839e+05,NaN,Administrative,1.582960e+08,2016-EU-TM-0117-M - INEA/CEF/TRAN/M2016/134961...,Innovation and Networks Executive Agency,Optimising the integration and interconnection...,Connecting Europe Facility (CEF),Grants,LU2 9PF,0.872252,fts_2017_42,2017
112,UNITED KINGDOM ATOMIC ENERGY AUTHORITY*UKAEA,no,5.736822e+06,NaN,Administrative,9.202316e+07,633053 - EUROFUSION - IMPLEMENTATION OF ACTIVI...,Directorate-General for Research and Innovation,Euratom — Fusion energy (08.03.01.01),Euratom Research and Training Programme,Grants,OX14 3DB,0.872252,fts_2017_112,2017
150,UNITED KINGDOM ATOMIC ENERGY AUTHORITY*UKAEA,no,6.000000e+07,NaN,Operational,6.000000e+07,NEW JOC CONTRACT FOR OPERATION OF JET FACILITI...,Directorate-General for Research and Innovation,Euratom — Fusion energy (08.03.01.01),Euratom Research and Training Programme,Grants,OX14 3DB,0.872252,fts_2017_150,2017


In [32]:
fts = pd.concat([fts_2016, fts_2017])
fts['amount_gbp'] = (fts.amount * fts.eur_gbp).round()
fts['total_amount_gbp'] = (fts.total_amount_eur * fts.eur_gbp).round()
fts.shape

(5408, 17)

### Erasmus

In [33]:
erasmus_organisations = pd.read_pickle('../erasmus/output/erasmus_mobility_organisations.pkl.gz')
erasmus_organisations.shape

(18883, 10)

In [34]:
erasmus_projects = pd.read_pickle('../erasmus/output/erasmus_mobility_projects.pkl.gz')
erasmus_projects.shape

(14274, 13)

In [35]:
erasmus = pd.merge(erasmus_projects, erasmus_organisations, on='project_identifier', validate='1:m')
erasmus.shape

(18855, 22)

In [36]:
validate_postcodes(erasmus)

erasmus['max_contribution_gbp'] = (erasmus.max_contribution_eur * erasmus.eur_gbp).round()
erasmus['my_eu_id'] = \
    'erasmus_' + erasmus.project_identifier + '_' + \
    erasmus.partner_number.apply('{:.0f}'.format).\
    str.replace('nan', 'coordinator', regex=False)
assert erasmus.shape[0] == erasmus.my_eu_id.unique().shape[0]
erasmus.head()

,project_identifier,funds,call_year,project,summary,project_status,max_contribution_eur,project_url,participating_countries,extra_partners,num_countries,num_organisations,eur_gbp,organisation_address,organisation_coordinator,organisation_country,organisation_name,partner_number,postcode,organisation_region,organisation_type,organisation_website,max_contribution_gbp,my_eu_id
0,2014-1-SK01-KA101-000041,Erasmus+,2014,Implementing CLIL method into the school curri...,The initial background of the Implementation C...,finalized,3005.0,NaN,"SK,UK",NaN,2,2,0.807325,"20 AVENUE ROAD, YO125JX, SCARBOROUGH",False,UK,ANGLOLANG (SCARBOROUGH) LTD,1.0,YO12 5JX,North Yorkshire,Other,http://www.anglolang.com,2426.0,erasmus_2014-1-SK01-KA101-000041_1
1,2014-1-LV01-KA101-000043,Erasmus+,2014,Skolotāju kompetences paaugstināšana,Context/background of project:\r\nNeretas Jāņa...,finalized,8805.0,NaN,"LV,DE,UK",NaN,3,3,0.807325,"The Towers, 38 Hawley Square, CT9 1PH, Margate",False,UK,English in Margate,2.0,CT9 1PH,NaN,Other,http://www.englishinmargate.com,7108.0,erasmus_2014-1-LV01-KA101-000043_2
2,2014-1-UK01-KA101-000044,Erasmus+,2014,Excellence in French teaching,Our school currently does some French teaching...,finalized,8840.0,NaN,UK,NaN,1,1,0.807325,"School Lane, CW12 4RG, Congleton",True,UK,Astbury St Mary's Primary School,NaN,CW12 4RG,Cheshire,School/Institute/Educational centre – General ...,http://www.astburyschool.org.uk,7137.0,erasmus_2014-1-UK01-KA101-000044_coordinator
3,2014-1-SE01-KA101-000044,Erasmus+,2014,Education for (E)U,Nösnäsgymnasiet has for many years sent studen...,finalized,39640.0,NaN,"SE,PL,UK,DE,CZ,IT",NaN,6,6,0.807325,"Science Park Campus, Kings Hedges Road, CB42QT...",False,UK,Cambridge Regional College,2.0,CB4 2QT,East Anglia,Other,http://www.camre.ac.uk,32002.0,erasmus_2014-1-SE01-KA101-000044_2
4,2014-1-UK01-KA101-000045,Erasmus+,2014,1 + 2 The World and You,Context and background:\r\nThe Scottish Govern...,finalized,1955.0,NaN,UK,NaN,1,1,0.807325,"67-69 Lochleven Road, KY5 8HU, Lochore",True,UK,Benarty Primary School,NaN,KY5 8HU,Eastern Scotland,School/Institute/Educational centre – General ...,NaN,1578.0,erasmus_2014-1-UK01-KA101-000045_coordinator


### NHS

In [37]:
nhs_staff = pd.read_pickle('../nhs/output/staff.pkl.gz')
nhs_hospital_postcodes = pd.read_pickle('../nhs/output/hospital_postcodes.pkl.gz')
validate_postcodes(nhs_hospital_postcodes)
[nhs_staff.shape, nhs_hospital_postcodes.shape, nhs_hospital_postcodes.hospital_organisation.nunique()]

[(229, 11), (880, 4), 880]

In [38]:
# dummy amount so we can put it in
nhs_hospital_postcodes['zero'] = 0

In [39]:
nhs_hospital_postcodes['my_eu_id'] = \
    'nhs_' + nhs_hospital_postcodes.hospital_organisation
nhs_hospital_postcodes.head()

,organisation,postcode,hospital_organisation,hospital_name,zero,my_eu_id
0,R0A,M13 9WL,R0A02,Manchester Royal Infirmary,0,nhs_R0A02
1,R0A,M13 9WL,R0A03,Royal Manchester Children's Hospital,0,nhs_R0A03
2,R0A,M13 9WL,R0A04,Manchester Royal Eye Hospital,0,nhs_R0A04
3,R0A,M13 9WL,R0A05,St Mary's Hospital,0,nhs_R0A05
8,R0A,M15 6FH,R0A33,University Dental Hospital,0,nhs_R0A33


### Interreg NW Europe

In [40]:
nweurope = pd.read_pickle('../nweurope/output/interreg_beneficiaries.pkl.gz')
validate_postcodes(nweurope)
nweurope.head(2)

,beneficiary,project,start_date,end_date,funding,union_cofinancing,category,last_update,postcode,eur_gbp,nwreg_benefs_id
4,Aberdeen City Council,ACE-Retrofitting - Accelerating Condominium En...,15.09.2016,14.03.2020,404388.20,0.6,013 - Energy efficiency renovation of public i...,23.10.2018,AB10 1AQ,0.872518,nwreg_benefs_4
8,Changeworks Resources for Life Ltd.,ACE-Retrofitting - Accelerating Condominium En...,15.09.2016,14.03.2020,117504.16,0.6,013 - Energy efficiency renovation of public i...,23.10.2018,EH6 5PY,0.872518,nwreg_benefs_8


### LIFE

In [41]:
life = pd.read_pickle('../life/output/life.pkl.gz')
validate_postcodes(life)

In [42]:
life['total_budget_gbp'] = (life.total_budget_eur * life.eur_gbp).round()
life['eu_contribution_gbp'] = (life.eu_contribution_eur * life.eur_gbp).round()
life.shape

(324, 21)

## Idea 1: All Points on Map, Data by District

This should make the map look fairly similar to how it looks now, so it seems like a good starting point.

In [43]:
ALL_PLACES = [
    (cordis, 'contribution_gbp', 'money'),
    (creative, 'max_contribution_gbp', 'money'),
    (esif, 'eu_investment', 'money'),
    (fts.drop('amount', axis=1), 'amount_gbp', 'money'),
    (erasmus, 'max_contribution_gbp', 'money'),
    (nhs_hospital_postcodes, 'zero', 'hospital'),
    (nweurope, 'funding', 'money'),
    (life, 'eu_contribution_gbp', 'money')
]

GeoJSON is very inefficient for representing a bunch of points, so let's use a relatively simple packed format.
```
min_longitude min_latitude
outcode incode delta_longitude delta_latitude incode delta_longitude delta_latitude
```
We need [about 4 decimal places](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude).

In [44]:
def add_outward_and_inward_codes(df):
    df['outward_code'] = df.postcode.str.split(' ').str[0]
    df['inward_code'] = df.postcode.str.split(' ').str[1]
    return df

def add_packed_icon_mask(df):
    def pack_icon_mask(icons):
        # So far we just have one bit in our mask; we may have more.
        if 'hospital' in icons:
            return 1
        else:
            return 0
    MASK_BITS = 1
    assert df.amount.max() < 2**(32 - MASK_BITS)
    df['icon_mask'] = df.icons.apply(pack_icon_mask).astype('uint32')
    df['packed_amount'] = np.bitwise_or(
        np.left_shift(df.amount, MASK_BITS), df.icon_mask
    )
    return df

def pack_geocoded_postcodes(dfs):
    all_postcode_amounts = pd.concat([
        df.rename(columns={amount_column: 'amount'}).assign(icons=icon)\
        [['postcode', 'amount', 'icons']]
        for df, amount_column, icon in dfs
    ])
    postcode_amounts = all_postcode_amounts.groupby('postcode').\
        aggregate({'amount': sum, 'icons': lambda icons: set(icons)})
    postcode_amounts.reset_index(inplace=True)
    postcode_amounts.amount = postcode_amounts.amount.astype('uint32')
    add_outward_and_inward_codes(postcode_amounts)
    add_packed_icon_mask(postcode_amounts)
    
    geocoded_postcodes = pd.merge(postcode_amounts, ukpostcodes, validate='1:1')
    
    min_longitude = geocoded_postcodes.longitude.min()
    min_latitude = geocoded_postcodes.latitude.min()
    
    geocoded_postcodes['delta_longitude'] = geocoded_postcodes.longitude - min_longitude
    geocoded_postcodes['delta_latitude'] = geocoded_postcodes.latitude - min_latitude
    
    return {
        'min_longitude': min_longitude,
        'min_latitude': min_latitude,
        'geocoded_postcodes': geocoded_postcodes
    }

packed_postcodes = pack_geocoded_postcodes(ALL_PLACES)
[
    packed_postcodes['min_longitude'],
    packed_postcodes['min_latitude'],
    packed_postcodes['geocoded_postcodes'].shape[0]
]

[-7.825484, 49.9124760541243, 9805]

In [45]:
packed_postcodes['geocoded_postcodes'].head()

,postcode,amount,icons,outward_code,inward_code,icon_mask,packed_amount,id,latitude,longitude,delta_longitude,delta_latitude
0,AB10 1AB,1131202,{money},AB10,1AB,0,2262404,1812462,57.149606,-2.096916,5.728568,7.237130
1,AB10 1AQ,1418530,{money},AB10,1AQ,0,2837060,1693233,57.148081,-2.094664,5.730820,7.235605
2,AB10 1FE,3054,{money},AB10,1FE,0,6108,1693258,57.148075,-2.101969,5.723515,7.235599
3,AB10 1FR,230178,{money},AB10,1FR,0,460356,1693263,57.148372,-2.101160,5.724324,7.235896
4,AB10 1FY,12098519,{money},AB10,1FY,0,24197038,1693267,57.148232,-2.096648,5.728836,7.235756


In [46]:
def make_packed_postcode_json(packed_postcodes):
    packed_postcodes = packed_postcodes.copy()
   
    grouped_postcodes = packed_postcodes['geocoded_postcodes'].\
        sort_values('outward_code').groupby('outward_code')
     
    def make_code_tuples(row):
        coordinate = '{0:.4f}'
        return [
            row['inward_code'],
            float(coordinate.format(row['delta_longitude'])),
            float(coordinate.format(row['delta_latitude'])),
            row['packed_amount']
        ]
    
    postcodes = {}
    for outward_code, group in grouped_postcodes:
        postcodes[outward_code] = [
            x
            for code in group.sort_values('inward_code').apply(make_code_tuples, axis=1)
            for x in code
        ]

    min_coordinate = '{0:.6f}'
    return {
        'min_longitude': float(min_coordinate.format(packed_postcodes['min_longitude'])),
        'min_latitude': float(min_coordinate.format(packed_postcodes['min_latitude'])),
        'postcodes': postcodes
    }

with open('output/packed_postcodes.data.json', 'w') as file:
    json.dump(make_packed_postcode_json(packed_postcodes), file, sort_keys=True)

### Some Aggregates

In [47]:
[place[0].shape[0] for place in ALL_PLACES]

[27293, 320, 1351, 5408, 18855, 880, 88, 324]

In [48]:
sum(place[0].shape[0] for place in ALL_PLACES)

54519

### Data by District

#### CORDIS

In [49]:
MAX_PROJECTS = 50

# Dump to JSON using pandas, because it puts in nulls instead of NaNs for
# missing values. Then load the JSON into dicts for 
def make_district_data_json(df, name):
    key = ['outwardCode', 'inwardCode']
    def to_json(group):
        group.drop(key, axis=1, inplace=True)
        return json.loads(group.to_json(orient='split', index=False))
    grouped = df.groupby(key).apply(to_json)
    
    grouped = grouped.reset_index()
    grouped.columns = key + [name]
    
    for _key, row in grouped.iterrows():
        count = len(row[name]['data'])
        if count > MAX_PROJECTS:
            row[name]['data'] = row[name]['data'][0:MAX_PROJECTS]
            row[name]['extra'] = count - MAX_PROJECTS
        
    return grouped

def make_cordis_district_data(cordis):
    cordis = add_outward_and_inward_codes(cordis.copy())

    cordis = cordis[[
        'outward_code',
        'inward_code',
        'start_date', 'end_date',
        'title',
        'name', # of organization
        'objective',
        'contribution_gbp',
        'total_cost_gbp',
        'num_countries',
        'num_organizations',
        'acronym',
        'project_url',
        'organization_url',
        'image_path',
        'my_eu_id'
    ]]
    
    cordis.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'start_date': 'startDate',
        'end_date': 'endDate',
        'title': 'projectTitle',
        'name': 'organisationName',
        'contribution_gbp': 'contribution',
        'total_cost_gbp': 'totalCost',
        'num_countries': 'numCountries',
        'num_organizations': 'numOrganisations',
        'project_url': 'projectUrl',
        'organization_url': 'organisationUrl',
        'image_path': 'imagePath',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    cordis.sort_values(
        ['outwardCode', 'inwardCode', 'contribution'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(cordis, 'cordis')

cordis_district_data = make_cordis_district_data(cordis)
cordis_district_data.head()

,outwardCode,inwardCode,cordis
0,AB10,1FY,"{'columns': ['startDate', 'endDate', 'projectT..."
1,AB10,1UQ,"{'columns': ['startDate', 'endDate', 'projectT..."
2,AB10,7QB,"{'columns': ['startDate', 'endDate', 'projectT..."
3,AB11,5SS,"{'columns': ['startDate', 'endDate', 'projectT..."
4,AB11,9DB,"{'columns': ['startDate', 'endDate', 'projectT..."


#### Creative Europe

In [50]:
def make_creative_district_data(creative):
    creative = add_outward_and_inward_codes(creative.copy())
    
    coordinators = creative[creative.organisation_coordinator]
    coordinators = coordinators[['project_number', 'organisation_name']]
    creative = pd.merge(
        creative, coordinators,
        how='left', on='project_number', suffixes=('', '_coordinator'))

    creative = creative[[
        'outward_code',
        'inward_code',
        'start_date', 'end_date',
        'project',
        'organisation_name',
        'max_contribution_gbp',
        'num_countries',
        'num_organisations',
        'summary',
        'organisation_website',
        'organisation_name_coordinator',
        'my_eu_id'
    ]]
    
    creative.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'start_date': 'startDate',
        'end_date': 'endDate',
        'organisation_name': 'organisationName',
        'max_contribution_gbp': 'maxContribution',
        'num_countries': 'numCountries',
        'num_organisations': 'numOrganisations',
        'organisation_website': 'organisationWebsite',
        'organisation_name_coordinator': 'coordinatorName',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    creative.sort_values(
        ['outwardCode', 'inwardCode', 'maxContribution'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(creative, 'creative')

creative_district_data = make_creative_district_data(creative)
creative_district_data.head()

,outwardCode,inwardCode,creative
0,B1,2EP,"{'columns': ['startDate', 'endDate', 'project'..."
1,B5,5JU,"{'columns': ['startDate', 'endDate', 'project'..."
2,B78,3HL,"{'columns': ['startDate', 'endDate', 'project'..."
3,BD9,4SA,"{'columns': ['startDate', 'endDate', 'project'..."
4,BN1,1AG,"{'columns': ['startDate', 'endDate', 'project'..."


#### ESIF

In [81]:
def make_esif_district_data(esif):
    esif = add_outward_and_inward_codes(esif.copy())
    esif = esif[[
        'outward_code',
        'inward_code',
        'start_date', 'end_date',
        'project',
        'beneficiary',
        'summary',
        'funds',
        'eu_investment',
        'project_cost',
        'my_eu_id'
    ]]
    
    esif.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'start_date': 'startDate',
        'end_date': 'endDate',
        'project': 'projectTitle',
        'beneficiary': 'organisationName',
        'eu_investment': 'euInvestment',
        'project_cost': 'projectCost',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    esif.sort_values(
        ['outwardCode', 'inwardCode', 'euInvestment'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(esif, 'esif')

esif_district_data = make_esif_district_data(esif)
esif_district_data.head()

KeyError: "['project' 'eu_investment' 'project_cost' 'my_eu_id'] not in index"

#### FTS

In [52]:
fts.columns

Index(['beneficiary', 'coordinator', 'amount', 'geographical_zone',
       'expense_type', 'total_amount_eur', 'subject_of_grant_or_contract',
       'responsible_department', 'budget_line_name_and_number', 'action_type',
       'funding_type', 'postcode', 'eur_gbp', 'my_eu_id', 'year', 'amount_gbp',
       'total_amount_gbp'],
      dtype='object')

In [53]:
def make_fts_district_data(fts):
    fts = add_outward_and_inward_codes(fts.copy())
    fts = fts[[
        'outward_code',
        'inward_code',
        'year',
        'beneficiary',
        'amount_gbp',
        'budget_line_name_and_number',
        'my_eu_id'
    ]]
    
    fts.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'amount_gbp': 'amount',
        'budget_line_name_and_number': 'budgetLineNameAndNumber',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    fts.sort_values(
        ['outwardCode', 'inwardCode', 'amount'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(fts, 'fts')

fts_district_data = make_fts_district_data(fts)
fts_district_data.head()

,outwardCode,inwardCode,fts
0,AB10,1FY,"{'columns': ['year', 'beneficiary', 'amount', ..."
1,AB10,7QB,"{'columns': ['year', 'beneficiary', 'amount', ..."
2,AB12,3BN,"{'columns': ['year', 'beneficiary', 'amount', ..."
3,AB15,4ZT,"{'columns': ['year', 'beneficiary', 'amount', ..."
4,AB23,8AQ,"{'columns': ['year', 'beneficiary', 'amount', ..."


#### Erasmus

In [54]:
def make_erasmus_district_data(erasmus):
    erasmus = add_outward_and_inward_codes(erasmus.copy())

    coordinators = erasmus[erasmus.organisation_coordinator]
    coordinators = coordinators[['project_identifier', 'organisation_name']]
    erasmus = pd.merge(
        erasmus, coordinators,
        how='left', on='project_identifier', suffixes=('', '_coordinator'))

    erasmus = erasmus[[
        'outward_code',
        'inward_code',
        'call_year',
        'project',
        'organisation_name',
        'max_contribution_gbp',
        'num_countries',
        'num_organisations',
        'summary',
        'organisation_website',
        'organisation_name_coordinator',
        'my_eu_id'
    ]]
    
    erasmus.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'call_year': 'callYear',
        'organisation_name': 'organisationName',
        'max_contribution_gbp': 'maxContribution',
        'num_countries': 'numCountries',
        'num_organisations': 'numOrganisations',
        'organisation_website': 'organisationWebsite',
        'organisation_name_coordinator': 'coordinatorName',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    erasmus.sort_values(
        ['outwardCode', 'inwardCode', 'maxContribution'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(erasmus, 'erasmus')

erasmus_district_data = make_erasmus_district_data(erasmus)
erasmus_district_data.head()

,outwardCode,inwardCode,erasmus
0,AB10,1FE,"{'columns': ['callYear', 'project', 'organisat..."
1,AB10,1FR,"{'columns': ['callYear', 'project', 'organisat..."
2,AB10,1FY,"{'columns': ['callYear', 'project', 'organisat..."
3,AB10,1JJ,"{'columns': ['callYear', 'project', 'organisat..."
4,AB10,7QB,"{'columns': ['callYear', 'project', 'organisat..."


#### NHS

Just store the organisation key for now. The data doesn't really fit with the district model.

In [55]:
def make_nhs_district_data(nhs_hospital_postcodes, nhs_staff):
    nhs = add_outward_and_inward_codes(pd.merge(
        nhs_hospital_postcodes, nhs_staff, on='organisation', validate='m:1'
    ))
    
    nhs.sort_values(
        ['outward_code', 'inward_code', 'eu_nurses'],
        ascending=[True, True, False],
        inplace=True
    )

    nhs = nhs[[
        'outward_code',
        'inward_code',
        'organisation',
        'hospital_name',
        'my_eu_id'
    ]]
    
    nhs.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'hospital_name': 'hospitalName',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    return make_district_data_json(nhs, 'nhs')

nhs_district_data = make_nhs_district_data(nhs_hospital_postcodes, nhs_staff)
nhs_district_data.head()

,outwardCode,inwardCode,nhs
0,AL3,5PN,"{'columns': ['organisation', 'hospitalName', '..."
1,AL5,4TA,"{'columns': ['organisation', 'hospitalName', '..."
2,AL6,9PW,"{'columns': ['organisation', 'hospitalName', '..."
3,AL7,4HQ,"{'columns': ['organisation', 'hospitalName', '..."
4,B11,4HL,"{'columns': ['organisation', 'hospitalName', '..."


#### Interreg NW Europe

In [56]:
nweurope.columns

Index(['beneficiary', 'project', 'start_date', 'end_date', 'funding',
       'union_cofinancing', 'category', 'last_update', 'postcode', 'eur_gbp',
       'nwreg_benefs_id'],
      dtype='object')

In [57]:
nweurope.head(2)

,beneficiary,project,start_date,end_date,funding,union_cofinancing,category,last_update,postcode,eur_gbp,nwreg_benefs_id
4,Aberdeen City Council,ACE-Retrofitting - Accelerating Condominium En...,15.09.2016,14.03.2020,404388.20,0.6,013 - Energy efficiency renovation of public i...,23.10.2018,AB10 1AQ,0.872518,nwreg_benefs_4
8,Changeworks Resources for Life Ltd.,ACE-Retrofitting - Accelerating Condominium En...,15.09.2016,14.03.2020,117504.16,0.6,013 - Energy efficiency renovation of public i...,23.10.2018,EH6 5PY,0.872518,nwreg_benefs_8


In [58]:
def make_nweurope_district_data(nweurope):
    nweurope = add_outward_and_inward_codes(nweurope.copy())
    nweurope = nweurope[[
        'outward_code',
        'inward_code',
        'beneficiary',
        'project',
        'start_date',
        'end_date',
        'funding',
        'union_cofinancing',
        'nwreg_benefs_id'
    ]]
    
    nweurope.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'start_date': 'startDate',
        'end_date': 'endDate',
        'union_cofinancing': 'unionCofinancing',
        'nwreg_benefs_id': 'myEuId'
    }, axis=1, inplace=True)
    
    nweurope.sort_values(
        ['outwardCode', 'inwardCode', 'unionCofinancing'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(nweurope, 'nweurope')

nweurope_district_data = make_nweurope_district_data(nweurope)
nweurope_district_data.head()

,outwardCode,inwardCode,nweurope
0,AB10,1AB,"{'columns': ['beneficiary', 'project', 'startD..."
1,AB10,1AQ,"{'columns': ['beneficiary', 'project', 'startD..."
2,B4,6BS,"{'columns': ['beneficiary', 'project', 'startD..."
3,B4,7XG,"{'columns': ['beneficiary', 'project', 'startD..."
4,BD13,3SY,"{'columns': ['beneficiary', 'project', 'startD..."


#### LIFE

In [59]:
life.columns

Index(['project_title', 'website', 'year', 'type_of_beneficiary',
       'project_url', 'project_id', 'Unnamed: 0_x', 'area_name', 'area_type',
       'postcode', 'clean_postcode', 'id', 'Unnamed: 0_y', 'coordinator',
       'total_budget_eur', 'eu_contribution_eur', 'background', 'eur_gbp',
       'my_eu_id', 'total_budget_gbp', 'eu_contribution_gbp'],
      dtype='object')

In [60]:
def make_life_district_data(life):
    life = add_outward_and_inward_codes(life.copy())
    life = life[[
        'outward_code',
        'inward_code',
        'year',
        'project_title',
        'coordinator',
        'total_budget_gbp',
        'eu_contribution_gbp',
        'background',
        'project_url',
        'website',
        'my_eu_id'
    ]]
    
    life.rename({
        'outward_code': 'outwardCode',
        'inward_code': 'inwardCode',
        'total_budget_gbp': 'amount',
        'eu_contribution_gbp': 'euContribution',
        'my_eu_id': 'myEuId'
    }, axis=1, inplace=True)
    
    life.sort_values(
        ['outwardCode', 'inwardCode', 'amount'],
        ascending=[True, True, False],
        inplace=True
    )
    
    return make_district_data_json(life, 'life')

life_district_data = make_life_district_data(life)
life_district_data.head()

,outwardCode,inwardCode,life
0,AB34,5EU,"{'columns': ['year', 'project_title', 'coordin..."
1,AB35,5TU,"{'columns': ['year', 'project_title', 'coordin..."
2,AB35,5UE,"{'columns': ['year', 'project_title', 'coordin..."
3,AB37,9ES,"{'columns': ['year', 'project_title', 'coordin..."
4,AB43,6QY,"{'columns': ['year', 'project_title', 'coordin..."


# Assemble full dataset in dataframe

Looking at each programme in turn, create a full_data_ dataframe with the key attributes:
postcode, projectTitle, euContribution, amount, website, myEuId

Check union_cofinancing same as euContribution

## Cordis

In [66]:
cordis.head(2)

,rcn,acronym,status,title,start_date,end_date,project_url,objective,total_cost_eur,max_contribution_eur,coordinator_country,participant_countries,brief_title,teaser,article,related_report_title,image_path,num_countries,num_organizations,eur_gbp,project_rcn,organization_id,role,name,contribution_eur,organization_url,postcode,my_eu_id,total_cost_gbp,max_contribution_gbp,contribution_gbp
0,210895,MODERN,ONG,"MOdeling and DEsign of Reliable, process varia...",2009-03-01,2012-02-01,NaN,The influence of process variations is becomin...,27363006.0,4385254.0,FR,ES;DK;AT;FR;CH;EL;IT;NL;DE;UK,NaN,NaN,NaN,NaN,NaN,10,26,0.871217,210895,999974165,participant,University of Glasgow,64128.0,http://www.gla.ac.uk,G12 8QQ,fp7_210895_999974165,23839127.0,3820510.0,55869.0
1,210893,CSSL,ONG,Consumerizing Solid State Lighting,2010-02-01,2013-01-01,NaN,Light-emitting diode (LED) lamps are a rapidly...,27092103.0,4524384.0,NL,BE;UK;NL;IT;FR;ES;CZ,NaN,NaN,NaN,NaN,NaN,7,22,0.846050,210893,966670282,participant,British Gas Trading Limited,14610.0,http://www.britishgas.co.uk,SL4 5GD,fp7_210893_966670282,22921264.0,3827854.0,12361.0


In [133]:
full_data_cordis = cordis[['postcode','title','contribution_gbp','total_cost_gbp','project_url','my_eu_id']].copy()

In [134]:
full_data_cordis.rename({
    'title': 'projectTitle',
    'contribution_gbp': 'euContribution',
    'total_cost_gbp': 'amount',
    'project_url': 'website',
    'my_eu_id': 'myEuId'
}, axis=1, inplace=True)

In [135]:
full_data_cordis.head(2)

,postcode,projectTitle,euContribution,amount,website,myEuId
0,G12 8QQ,"MOdeling and DEsign of Reliable, process varia...",55869.0,23839127.0,NaN,fp7_210895_999974165
1,SL4 5GD,Consumerizing Solid State Lighting,12361.0,22921264.0,NaN,fp7_210893_966670282


## Creative Europe

In [136]:
creative.head(2)

,project_number,funds,category,action,activity_type,call_year,start_date,end_date,project,summary,project_status,max_contribution_eur,is_success,project_url,results_available,results_url,participating_countries,extra_partners,num_countries,num_organisations,eur_gbp,organisation_address,organisation_coordinator,organisation_country,organisation_name,partner_number,postcode,organisation_region,organisation_type,organisation_website,max_contribution_gbp,my_eu_id
0,552179-CREA-1-2014-1-IT-CULT-COOP1,Creative Europe,Culture,Cooperation projects,NaN,2014,2014-11-03,2016-12-15,Virtual Sets: Creating and promoting virtual s...,ViSet will promote cutting-edge digital techno...,finalized,199440.7,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"IT,ES,UK,RS",NaN,4,5,0.769165,BASECAMP LIVERPOOL UNIT N BALTIC CREATIVE CAMP...,False,UK,PACIFICSTREAM ENTERPRISE SOLUTIONSCOMMUNITY IN...,2.0,L1 0AH,Merseyside,Other,http://pacifcstreamenterprise.co.uk,153403.0,creative_552179-CREA-1-2014-1-IT-CULT-COOP1_2
1,552036-CREA-1-2014-1-IT-CULT-COOP2,Creative Europe,Culture,Cooperation projects,NaN,2014,2014-10-01,2018-09-30,"SMALL SIZE, PERFORMING ARTS FOR EARLY YEARS",The project has a precise focus: the diffusion...,ongoing,1995000.0,No,NaN,No,https://ec.europa.eu/programmes/creative-europ...,"IT,IE,DE,RO,HU,SE,FR,SI,DK,ES,UK,PL,AT,FI,BE",NaN,15,17,0.816518,"240 THE BROADWAY, SW19 1SB, LONDON",False,UK,POLKA CHILDRENS THEATRE LIMITED,11.0,SW19 1SB,Extra-Regio NUTS 2,Theatre,http://www.polkatheatre.com,1628953.0,creative_552036-CREA-1-2014-1-IT-CULT-COOP2_11


In [137]:
full_data_creative = creative[['postcode','max_contribution_gbp','organisation_website','my_eu_id']].copy()

In [138]:
full_data_creative.rename({
    'project': 'projectTitle',
    'max_contribution_gbp': 'euContribution',
    'organisation_website': 'website',
    'my_eu_id': 'myEuId'
}, axis=1, inplace=True)

In [139]:
full_data_creative.head(2)

,postcode,euContribution,website,myEuId
0,L1 0AH,153403.0,http://pacifcstreamenterprise.co.uk,creative_552179-CREA-1-2014-1-IT-CULT-COOP1_2
1,SW19 1SB,1628953.0,http://www.polkatheatre.com,creative_552036-CREA-1-2014-1-IT-CULT-COOP2_11


## ESIF

In [140]:
esif.head(2)

,beneficiary,project,funds,summary,start_date,end_date,eu_investment,project_cost,postcode,my_eu_id
0,Building Research Establishment Limited,Hertfordshire Low Carbon Innovation Network,ERDF,Strategic Fit and Value for Money requirements...,2016-05-01,2019-02-28,630000.0,1260000.0,WD25 9XX,erdf_england_0
1,GCGP LEP,Promoting Growth in Greater Cambridge Greater ...,ERDF,The project will provide local facilitation re...,2015-10-01,2018-12-31,373677.0,747356.0,PE28 4WX,erdf_england_1


In [141]:
full_data_esif = esif[['postcode','project','eu_investment','project_cost','my_eu_id']].copy()

In [142]:
full_data_esif.rename({
    'project': 'projectTitle',
    'eu_investment': 'euContribution',
    'project_cost': 'amount',
    'my_eu_id': 'myEuId'
}, axis=1, inplace=True)

In [143]:
full_data_esif.head(2)

,postcode,projectTitle,euContribution,amount,myEuId
0,WD25 9XX,Hertfordshire Low Carbon Innovation Network,630000.0,1260000.0,erdf_england_0
1,PE28 4WX,Promoting Growth in Greater Cambridge Greater ...,373677.0,747356.0,erdf_england_1


## FTS

In [144]:
fts.head(2)

,beneficiary,coordinator,amount,geographical_zone,expense_type,total_amount_eur,subject_of_grant_or_contract,responsible_department,budget_line_name_and_number,action_type,funding_type,postcode,eur_gbp,my_eu_id,year,amount_gbp,total_amount_gbp
339,NATS (EN ROUTE) PUBLIC LIMITED COMPANY*NERL,yes,7.450001e+07,NaN,Operational,74500011.05,2015-EU-TM-0193-M - NO MOVE/E2-2014-717/SESAR...,Innovation and Networks Executive Agency,Optimising the integration and interconnection...,Connecting Europe Facility (CEF),Grants,PO15 7FL,0.811893,fts_2016_339,2016,60486003.0,60486003.0
367,UNITED KINGDOM ATOMIC ENERGY AUTHORITY*UKAEA,no,4.673079e+06,NaN,Administrative,72239003.00,633053 - EUROFUSION - IMPLEMENTATION OF ACTIVI...,Directorate-General for Research and Innovation,Euratom — Fusion energy (08.03.01.01),Euratom Research and Training Programme,Grants,OX14 3DB,0.811893,fts_2016_367,2016,3794038.0,58650308.0


In [145]:
full_data_fts = fts[['postcode','budget_line_name_and_number','total_amount_gbp','amount_gbp','my_eu_id']].copy()

In [146]:
full_data_fts.rename({
    'budget_line_name_and_number': 'projectTitle',
    'total_amount_gbp': 'amount',
    'amount_gbp': 'euContribution',
    'my_eu_id': 'myEuId'
}, axis=1, inplace=True)

In [147]:
full_data_fts.head(2)

,postcode,projectTitle,amount,euContribution,myEuId
339,PO15 7FL,Optimising the integration and interconnection...,60486003.0,60486003.0,fts_2016_339
367,OX14 3DB,Euratom — Fusion energy (08.03.01.01),58650308.0,3794038.0,fts_2016_367


## Erasmus

In [148]:
full_data_cordis.head(2)

,postcode,projectTitle,euContribution,amount,website,myEuId
0,G12 8QQ,"MOdeling and DEsign of Reliable, process varia...",55869.0,23839127.0,NaN,fp7_210895_999974165
1,SL4 5GD,Consumerizing Solid State Lighting,12361.0,22921264.0,NaN,fp7_210893_966670282


In [149]:
erasmus.head(2)

,project_identifier,funds,call_year,project,summary,project_status,max_contribution_eur,project_url,participating_countries,extra_partners,num_countries,num_organisations,eur_gbp,organisation_address,organisation_coordinator,organisation_country,organisation_name,partner_number,postcode,organisation_region,organisation_type,organisation_website,max_contribution_gbp,my_eu_id
0,2014-1-SK01-KA101-000041,Erasmus+,2014,Implementing CLIL method into the school curri...,The initial background of the Implementation C...,finalized,3005.0,NaN,"SK,UK",NaN,2,2,0.807325,"20 AVENUE ROAD, YO125JX, SCARBOROUGH",False,UK,ANGLOLANG (SCARBOROUGH) LTD,1.0,YO12 5JX,North Yorkshire,Other,http://www.anglolang.com,2426.0,erasmus_2014-1-SK01-KA101-000041_1
1,2014-1-LV01-KA101-000043,Erasmus+,2014,Skolotāju kompetences paaugstināšana,Context/background of project:\r\nNeretas Jāņa...,finalized,8805.0,NaN,"LV,DE,UK",NaN,3,3,0.807325,"The Towers, 38 Hawley Square, CT9 1PH, Margate",False,UK,English in Margate,2.0,CT9 1PH,NaN,Other,http://www.englishinmargate.com,7108.0,erasmus_2014-1-LV01-KA101-000043_2


In [150]:
full_data_erasmus = erasmus[['postcode','project','max_contribution_gbp','organisation_website','my_eu_id']].copy()

In [151]:
full_data_erasmus.rename({
    'project': 'projectTitle',
    'max_contribution_gbp': 'euContribution',
    'organisation_website' : 'website',
    'my_eu_id': 'myEuId'
}, axis=1, inplace=True)

In [152]:
full_data_erasmus.head(2)

,postcode,projectTitle,euContribution,website,myEuId
0,YO12 5JX,Implementing CLIL method into the school curri...,2426.0,http://www.anglolang.com,erasmus_2014-1-SK01-KA101-000041_1
1,CT9 1PH,Skolotāju kompetences paaugstināšana,7108.0,http://www.englishinmargate.com,erasmus_2014-1-LV01-KA101-000043_2


## NWEurope

In [153]:
nweurope.head(2)

,beneficiary,project,start_date,end_date,funding,union_cofinancing,category,last_update,postcode,eur_gbp,nwreg_benefs_id
4,Aberdeen City Council,ACE-Retrofitting - Accelerating Condominium En...,15.09.2016,14.03.2020,404388.20,0.6,013 - Energy efficiency renovation of public i...,23.10.2018,AB10 1AQ,0.872518,nwreg_benefs_4
8,Changeworks Resources for Life Ltd.,ACE-Retrofitting - Accelerating Condominium En...,15.09.2016,14.03.2020,117504.16,0.6,013 - Energy efficiency renovation of public i...,23.10.2018,EH6 5PY,0.872518,nwreg_benefs_8


In [154]:
full_data_nweurope = nweurope[['postcode','project','funding','union_cofinancing','nwreg_benefs_id']].copy()

In [155]:
full_data_nweurope['euContribution'] = full_data_nweurope.apply(lambda row: row['funding'] * row['union_cofinancing'], axis=1)

In [156]:
full_data_nweurope.rename({
    'project': 'projectTitle',
    'funding': 'amount',
    'nwreg_benefs_id': 'myEuId'
}, axis=1, inplace=True)

In [157]:
full_data_nweurope = full_data_nweurope.drop(['union_cofinancing'], axis=1)

In [158]:
full_data_nweurope.head(2)

,postcode,projectTitle,amount,myEuId,euContribution
4,AB10 1AQ,ACE-Retrofitting - Accelerating Condominium En...,404388.20,nwreg_benefs_4,242632.920
8,EH6 5PY,ACE-Retrofitting - Accelerating Condominium En...,117504.16,nwreg_benefs_8,70502.496


## LIFE

In [167]:
life.head(2)

,project_title,website,year,type_of_beneficiary,project_url,project_id,Unnamed: 0_x,area_name,area_type,postcode,clean_postcode,id,Unnamed: 0_y,coordinator,total_budget_eur,eu_contribution_eur,background,eur_gbp,my_eu_id,total_budget_gbp,eu_contribution_gbp
0,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,0.0,Elenydd - Mallaen,SPA,SY25 6ET,SY25 6ET,226626,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_0,7723338.0,4634003.0
1,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,1.0,Eryri/ Snowdonia,SCI,LL24 0EU,LL24 0EU,894708,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_1,7723338.0,4634003.0


In [168]:
full_data_life = life[['postcode','project_title','total_budget_gbp','eu_contribution_gbp','project_url','my_eu_id']].copy()

In [169]:
full_data_life.rename({
        'project_title': 'projectTitle',
        'total_budget_gbp': 'amount',
        'eu_contribution_gbp': 'euContribution',
        'project_url': 'website',
        'my_eu_id': 'myEuId'
}, axis=1, inplace=True)

In [170]:
full_data_life.head(2)

,postcode,projectTitle,amount,euContribution,website,myEuId
0,SY25 6ET,Celtic Rainforests LIFE - Restoring the Celtic...,7723338.0,4634003.0,http://ec.europa.eu/environment/life/project/P...,life_0
1,LL24 0EU,Celtic Rainforests LIFE - Restoring the Celtic...,7723338.0,4634003.0,http://ec.europa.eu/environment/life/project/P...,life_1


In [172]:
full_data = pd.concat([full_data_cordis,
                       full_data_creative,
                       full_data_esif,
                       full_data_fts,
                       full_data_erasmus,
                       full_data_nweurope,
                       full_data_life], sort=False)

In [173]:
full_data.shape

(53639, 6)

Location on myeu.uk website

In [202]:
full_data['myeuURL'] = full_data.apply(
    lambda row: 'https://www.myeu.uk/postcode/' + "/".join( row['postcode'].split() ), axis=1)

In [203]:
full_data.head(2)

,postcode,projectTitle,euContribution,amount,website,myEuId,myeuURL
0,G12 8QQ,"MOdeling and DEsign of Reliable, process varia...",55869.0,23839127.0,NaN,fp7_210895_999974165,https://www.myeu.uk/postcode/G12/8QQ
1,SL4 5GD,Consumerizing Solid State Lighting,12361.0,22921264.0,NaN,fp7_210893_966670282,https://www.myeu.uk/postcode/SL4/5GD


In [206]:
full_data.to_csv('../constituencyprojects/output/full_data.csv', index=False)

### Save Data

```
districtData.postcodes['A11'].cordis.data
districtData.postcodes['A11'].cordis.extra
```

In [62]:
ALL_DISTRICT_DATA = reduce(
    lambda left, right: pd.merge(left, right, on=('outwardCode', 'inwardCode'), how='outer'), [
    cordis_district_data,
    creative_district_data,
    esif_district_data,
    fts_district_data,
    erasmus_district_data,
    nhs_district_data,
    nweurope_district_data,
    life_district_data    
])
ALL_DATASET_NAMES = list(set(ALL_DISTRICT_DATA.columns) - set(['outwardCode', 'inwardCode']))
    
def save_columns():
    def find_columns(data):
        return data[~data.isna()].iloc[0]['columns']
  
    columns = {
        dataset_name: find_columns(ALL_DISTRICT_DATA[dataset_name])
        for dataset_name in ALL_DATASET_NAMES
    }
    
    with open(os.path.join('output', 'district_columns.json'), 'w') as file:
        json.dump(columns, file, sort_keys=True)
save_columns()

In [66]:
life.head()

,project_title,website,year,type_of_beneficiary,project_url,project_id,Unnamed: 0_x,area_name,area_type,postcode,clean_postcode,id,Unnamed: 0_y,coordinator,total_budget_eur,eu_contribution_eur,background,eur_gbp,my_eu_id,total_budget_gbp,eu_contribution_gbp
0,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,0.0,Elenydd - Mallaen,SPA,SY25 6ET,SY25 6ET,226626,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_0,7723338.0,4634003.0
1,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,1.0,Eryri/ Snowdonia,SCI,LL24 0EU,LL24 0EU,894708,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_1,7723338.0,4634003.0
2,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,2.0,Coedydd Derw a Safleoedd Ystlumod Meirion/ Mei...,SCI,LL40 2HW,LL40 2HW,890808,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_2,7723338.0,4634003.0
3,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,3.0,Coed Cwm Einion,SCI,SY20 8TE,SY20 8TE,229335,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_3,7723338.0,4634003.0
4,Celtic Rainforests LIFE - Restoring the Celtic...,NaN,2017,Park-Reserve authority,http://ec.europa.eu/environment/life/project/P...,6690,4.0,Cwm Doethie - Mynydd Mallaen,SCI,SA20 0PN,SA20 0PN,411512,1,Snowdonia National Park Authority,9512759.0,5707655.0,The Celtic Rainforests are regarded as the ‘te...,0.811893,life_4,7723338.0,4634003.0


In [ ]:
ALL_DATASET_NAMES

In [ ]:
ALL_DISTRICT_DATA[(~ALL_DISTRICT_DATA[ALL_DATASET_NAMES].isna()).sum(axis=1) > 4][['outwardCode', 'inwardCode']].head()

In [ ]:
def merge_district_data(all_data):
    def find_dataset_data(row):
        dataset_data = row.drop(['outwardCode', 'inwardCode']).dropna().to_dict()
        return {
            dataset: { k: v for k, v in json.items() if k != 'columns' }
            for dataset, json in dataset_data.items()
        }
    
    def make_merged_district_data(outward_code, group):
        return {
            'outwardCode': outward_code,
            'postcodes': {
                row.inwardCode: find_dataset_data(row)
                for _index, row in group.iterrows()
            }
        }
    
    return {
        outward_code: make_merged_district_data(outward_code, group)
        for outward_code, group in all_data.groupby('outwardCode')
    }

district_data = merge_district_data(ALL_DISTRICT_DATA)
district_data['CA4']

In [ ]:
OUTPUT_DISTRICT_PATH = 'output/district'

def list_district_data(path):
    return glob.glob(os.path.join(path, '*.data.json'))

def clear_district_data(path):
    for f in list_district_data(path):
        os.remove(f)

def write_district_data(district_data, path):
    os.makedirs(path, exist_ok=True)
    clear_district_data(path)
    for outward_code, data in district_data.items():
        output_pathname = os.path.join(path, outward_code + '.data.json')
        with open(output_pathname, 'w') as file:
            json.dump(data, file, sort_keys=True)
write_district_data(district_data, OUTPUT_DISTRICT_PATH)

In [ ]:
def find_district_data_stats():
    files = list_district_data(OUTPUT_DISTRICT_PATH)
    return pd.DataFrame({
        'file': [file for file in files],
        'byte_size': [os.stat(file).st_size for file in files]
    })
district_data_stats = find_district_data_stats()
district_data_stats.describe()

In [ ]:
district_data_stats.byte_size.sum() / 1024 / 1024

In [ ]:
district_data_stats[district_data_stats.byte_size > 1024*1024]

In [ ]:
district_data_stats.describe().hist()

#### Data Index

Generate a JS file that webpack can use to make paths for all of the data files.

In [ ]:
def write_district_data_js():
    data_files = list_district_data(OUTPUT_DISTRICT_PATH)
    
    def make_require(data_file):
        basename = os.path.basename(data_file)
        pathname = os.path.join('.', 'district', basename)
        outward_code = basename.split('.')[0]
        return "  {}: require('{}')".format(outward_code, pathname)

    with open('output/district.js', 'w') as file:
        file.write('// NB: This file is generated automatically. Do not edit.\n')
        file.write('export default {\n')
        requires = [
            make_require(data_file)
            for data_file in sorted(data_files)
        ]
        file.write(',\n'.join(requires))
        file.write('\n}\n')
write_district_data_js()

#### NHS Data

In [ ]:
[nhs_staff.shape[0], nhs_hospital_postcodes.shape[0]]

In [ ]:
nhs_hospital_postcodes.groupby('hospital_organisation').organisation.count().max()

In [ ]:
def write_nhs_staff_data():
    with open('output/nhs_staff.json', 'w') as file:
        file.write(
            nhs_staff.sort_values('organisation', ascending=True).\
            to_json(orient='split', index=False))
write_nhs_staff_data()